In [26]:
import pandas as pd
#import matplotlib.pyplot as plt
#import seaborn as sns
import numpy as np
#import re
#from sklearn.ensemble import RandomForestClassifier
#np.random.seed(0)
#sns.set()
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from google.colab import drive
drive.mount("/content/gdrive")
!pwd

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content


In [27]:
df_test = pd.read_csv('/content/gdrive/MyDrive/InteligenciaArtificial/Titanic/test.csv')
df_train = pd.read_csv('/content/gdrive/MyDrive/InteligenciaArtificial/Titanic/train.csv')
df_gender_submission = pd.read_csv('/content/gdrive/MyDrive/InteligenciaArtificial/Titanic/gender_submission.csv')

In [28]:
features = pd.read_csv('/content/gdrive/MyDrive/InteligenciaArtificial/Titanic/new_train_mod.csv')
features = features[['Pclass','Sex','Age','SibSp','Parch','Fare','Cabin','Embarked_S','Embarked_C','Embarked_Q']]

features

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked_S,Embarked_C,Embarked_Q
0,3,1,22.0,1,0,7.2500,0,1,0,0
1,1,0,38.0,1,0,71.2833,1,0,1,0
2,3,0,26.0,0,0,7.9250,0,1,0,0
3,1,0,35.0,1,0,53.1000,1,1,0,0
4,3,1,35.0,0,0,8.0500,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...
886,2,1,27.0,0,0,13.0000,0,1,0,0
887,1,0,19.0,0,0,30.0000,1,1,0,0
888,3,0,21.0,1,2,23.4500,0,1,0,0
889,1,1,26.0,0,0,30.0000,1,0,1,0


In [29]:
featuresTest = pd.read_csv('/content/gdrive/MyDrive/InteligenciaArtificial/Titanic/new_test_mod.csv')
featuresTest = featuresTest[['Pclass','Sex','Age','SibSp','Parch','Fare','Cabin','Embarked_S','Embarked_C','Embarked_Q']]

# DecisionTreeClassifier

In [30]:
scaler = StandardScaler()
X_data = scaler.fit_transform(features)
y_data = pd.factorize(df_train['Survived'])[0]

X_test = featuresTest
scaler = StandardScaler()
X_data_test = scaler.fit_transform(X_test)

In [31]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

In [46]:
decisionTree = DecisionTreeClassifier(max_depth=3)
decisionTree.fit(X_data, y_data)

DecisionTreeClassifier(max_depth=3)

In [45]:
df_final = pd.DataFrame(df_gender_submission['PassengerId'])
df_final['Survived'] = y_pred
df_final.to_csv('/content/gdrive/MyDrive/InteligenciaArtificial/Titanic/predictionsTree.csv', index=False)
decisionTree.score(X_data, y_data)

0.819304152637486

# RandomForest

In [34]:
scaler = StandardScaler()
X_data = scaler.fit_transform(features)
y_data = pd.factorize(df_train['Survived'])[0]

X_test = featuresTest
scaler = StandardScaler()
X_data_test = scaler.fit_transform(X_test)

In [35]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()

param_grid = [{'n_estimators': [10, 100, 200, 500], 'max_depth': [None,5,10],
              'min_samples_split': [2,3,4], 'n_jobs': [10]}]

grid_search = GridSearchCV(clf, param_grid, cv=3, scoring='accuracy', return_train_score=True )

grid_search.fit(X_data, y_data)

GridSearchCV(cv=3, estimator=RandomForestClassifier(),
             param_grid=[{'max_depth': [None, 5, 10],
                          'min_samples_split': [2, 3, 4],
                          'n_estimators': [10, 100, 200, 500],
                          'n_jobs': [10]}],
             return_train_score=True, scoring='accuracy')

In [36]:
final_clf = grid_search.best_estimator_
final_clf

RandomForestClassifier(max_depth=5, n_estimators=200, n_jobs=10)

In [44]:
featuresTest = pd.read_csv('/content/gdrive/MyDrive/InteligenciaArtificial/Titanic/new_test_mod.csv')
featuresTest = featuresTest[['Pclass','Sex','Age','SibSp','Parch','Fare','Cabin','Embarked_S','Embarked_C','Embarked_Q']]

X_test = featuresTest

scaler = StandardScaler()
X_data_test = scaler.fit_transform(X_test)
predictions = final_clf.predict(X_data_test)

df_final = pd.DataFrame(df_gender_submission['PassengerId'])
df_final['Survived'] = predictions
df_final.to_csv('/content/gdrive/MyDrive/InteligenciaArtificial/Titanic/predictions.csv', index=False)
final_clf.score(X_data, y_data)

0.8608305274971941

# XGBoost

In [38]:
scaler = StandardScaler()
X_data = scaler.fit_transform(features)
y_data = pd.factorize(df_train['Survived'])[0]

X_test = featuresTest
scaler = StandardScaler()
X_data_test = scaler.fit_transform(X_test)

In [39]:
import xgboost
from sklearn.model_selection import GridSearchCV

In [40]:
xgb = xgboost.XGBClassifier()
parameters = {'objective': ['binary:logistic'],
              'max_depth': [None,5,7,10],
              'learning_rate': [0.05, 0.001,0.01],
              'n_estimators': [10, 100, 200, 500],
              'min_samples_split': [2,3,4]}
fit_params = {'early_stopping_rounds': 10,
              'eval_metric': 'logloss',
              'eval_set': [(X_data, y_data)]}
grid_searchXGB = GridSearchCV(xgb, parameters, cv=3, scoring='accuracy', n_jobs=10)
grid_searchXGB.fit(X_data, y_data, **fit_params)

/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:835: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[07:23:18] WARNING: ../src/learner.cc:767: 
Parameters: { "min_samples_split" } are not used.

[0]	validation_0-logloss:0.68766
[1]	validation_0-logloss:0.68228
[2]	validation_0-logloss:0.67700
[3]	validation_0-logloss:0.67182
[4]	validation_0-logloss:0.66680
[5]	validation_0-logloss:0.66187
[6]	validation_0-logloss:0.65705
[7]	validation_0-logloss:0.65230
[8]	validation_0-logloss:0.64766
[9]	validation_0-logloss:0.64310
[10]	validation_0-logloss:0.63862
[11]	validation_0-logloss:0.63422
[12]	validation_0-logloss:0.62990
[13]	validation_0-logloss:0.62565
[14]	validation_0-logloss:0.62148
[15]	validation_0-logloss:0.61735
[16]	validation_0-logloss:0.61329
[17]	validation_0-logloss:0.60933
[18]	validation_0-logloss:0.60540
[19]	validation_0-logloss:0.60152
[20]	validation_0-logloss:0.59773
[21]	validation_0-logloss:0.59400
[22]	validation_0-logloss:0.59034
[23]	validation_0-logloss:0.58676
[24]	validation_0-logloss:0.58322
[25]	validation_0-logloss:0.57970
[26]	validation_0-logloss:0.576

GridSearchCV(cv=3,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     gpu_id=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None, predictor=None,
                                     random_state=None, ...),
             n_jobs=10,
             param_grid={'learning_rate': [0.05, 0.001, 0.01],
                         'max_depth': [None, 5, 7, 10],
                         'min_samples_split': [2, 3, 4],
                         'n_estimators': [10, 100, 200, 500],
                         'objective': ['binary:logistic']},
             scoring='accuracy')

In [41]:
best_xgb = grid_searchXGB.best_estimator_
best_xgb

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.01, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, min_samples_split=2, missing=nan,
              monotone_constraints=None, n_estimators=500, n_jobs=None,
              num_parallel_tree=None, predictor=None, ...)

In [43]:
X_data_test = scaler.fit_transform(X_test)
predictionsXGB = best_xgb.predict(X_data_test)
df_final = pd.DataFrame(df_gender_submission['PassengerId'])
df_final['Survived'] = predictionsXGB
df_final.to_csv('/content/gdrive/MyDrive/InteligenciaArtificial/Titanic/predictionsXGB.csv', index=False)
best_xgb.score(X_data, y_data)

0.9023569023569024